In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py 
import plotly.express as px
import numpy as np

Data Preparation
1.	Load the student marks and attendance data from the provided Excel files into two separate Pandas DataFrames.


In [ ]:
student_marks=pd.read_excel("Student Performance Analysis.xlsx",sheet_name="Marks")
student_marks.head()

In [ ]:
student_attendance=pd.read_excel("Student Performance Analysis.xlsx",sheet_name="Attendance")
student_attendance.head()

2.	Merge the two DataFrames on the Name column to create a single DataFrame.

In [ ]:
student_ma_merged=student_marks.merge(student_attendance,how="inner",on="Name")
student_ma_merged

3.	Replace attendance values ('Y' or 'N') with numeric values (1 for 'Y' and 0 for 'N').

In [ ]:
student_ma_merged.replace("Y",1,inplace=True)

In [ ]:
student_ma_merged.replace("N",0,inplace=True)

4.	Deal with the missing values.

In [ ]:
student_ma_merged.isna().sum()

In [ ]:
student_ma_merged.fillna(0,inplace=True)

In [ ]:
student_ma_merged.isna().sum()

5.	Do the column cleaning like name column is the combination of upper and lower case.

In [ ]:
student_ma_merged.Name=student_ma_merged.Name.str.title()

In [ ]:
print(list(student_ma_merged.Name))

In [ ]:
student_ma_merged.describe(include="object").T

In [ ]:
student_ma_merged.describe(include=["int","float"]).T

In [ ]:
student_ma_merged.replace(" ","0",inplace=True)

In [ ]:
student_ma_merged['Mini Test 1']=pd.to_numeric(student_ma_merged['Mini Test 1'])

In [ ]:
student_ma_merged['Mini Test 2']=pd.to_numeric(student_ma_merged['Mini Test 2'])

In [ ]:
student_ma_merged['Live Test']=pd.to_numeric(student_ma_merged['Live Test'])

In [ ]:
student_ma_merged.loc[(student_ma_merged["Live Test"]>20),["Live Test"]]=20

In [ ]:
student_ma_merged.describe().T

In [ ]:
student_ma_merged.describe(include="object").T

In [ ]:
student_ma_merged["Assignment"]=student_ma_merged["Assignment"].astype(np.int8)

1.	Create separate columns for below:
    1. Total marks for each student.
    2. Percentage marks for each student.
    3. Attendance percentage for each student based on attendance columns.
    4. Calculate the weighted percentage – Attendance(40%), Mini test01(10%), Mini test02(10%), Live test(20%), Assignment (20%).
    5. Use this weighted percentage for further calculations.


In [ ]:
student_ma_merged["Total_Marks"]=round(student_ma_merged["Mini Test 1"] + student_ma_merged["Mini Test 2"] +\
                                       student_ma_merged["Live Test"] + student_ma_merged["Assignment"],0)

In [ ]:
student_ma_merged.head()

In [ ]:
student_ma_merged["percentage"]=round((student_ma_merged["Total_Marks"]/80)*100,2)
student_ma_merged.head()

In [ ]:
student_ma_merged["Attendance percentage"]=round(((student_ma_merged["Attendance Day 1"]+student_ma_merged["Attendance Day 2"]+\
student_ma_merged["Attendance Day 3"]+student_ma_merged["Attendance Day 4"]+student_ma_merged["Attendance Day 5"])/5)*100,2)
student_ma_merged.head()

In [ ]:
student_ma_merged["weighted percentage"]=(0.40*student_ma_merged["Attendance percentage"]+0.10*((student_ma_merged["Mini Test 1"]/20)*100)+\
0.10*((student_ma_merged["Mini Test 2"]/20)*100)+0.20*((student_ma_merged["Live Test"]/20)*100)+0.20*((student_ma_merged["Assignment"]/20)*100))
student_ma_merged.head()

In [ ]:
def performance(n):
    if n>=85:
        return "Excellent"
    elif n<=84 and n>=71:
        return "Good"
    elif n<=70 and n>=50:
        return "Average"
    elif n<50:
        return "Needs Improvement"

In [ ]:
student_ma_merged["performance"]=student_ma_merged["weighted percentage"].map(performance)
student_ma_merged.head()

Analysis 
1. Identify students with attendance below 75% but weighted percentage >50%.
2. Highlight the top three students based on percentage marks.
3. Impact of attendance on Tests/Assignment marks. 


In [ ]:
student_ma_merged.loc[(student_ma_merged["Attendance percentage"]<75) & (student_ma_merged["weighted percentage"]>50),["Name"]]

In [ ]:
student_ma_merged.sort_values(by="weighted percentage",ascending=False).head(3)

3.	Impact of attendance on Tests/Assignment marks

In [ ]:
student_ma_merged["Total_Attendance"]=student_ma_merged["Attendance Day 1"]+student_ma_merged["Attendance Day 2"]+\
student_ma_merged["Attendance Day 3"]+student_ma_merged["Attendance Day 4"]+student_ma_merged["Attendance Day 5"]
student_ma_merged["Test_Marks"]=student_ma_merged["Mini Test 1"] + student_ma_merged["Mini Test 2"] +student_ma_merged["Live Test"]
student_ma_merged.head()

In [ ]:
corr_marks_test=round(student_ma_merged["Total_Attendance"].corr(student_ma_merged["Test_Marks"]),4)
corr_marks_live=round(student_ma_merged["Total_Attendance"].corr(student_ma_merged["Live Test"]),4)
corr_marks_total=round(student_ma_merged["Total_Attendance"].corr(student_ma_merged["Total_Marks"]),4)

In [ ]:
print("The correlation b/w total attendance and total test (exlcuding assignments marks) marks",corr_marks_test)
print("The correlation b/w total attendance and Lives tests only",corr_marks_live)
print("The correlation b/w total attendance and Total Marks",corr_marks_total)

#### We can safely say that there is no corelation b/w marks and attendance.

Visualization 
1. Create a bar chart displaying weighted percentages for top 5 students.
2.	Create a pie chart showing the distribution of students across the four performance categories.
3.	Create box plots for each test (Live Test, Mini Test 1, Mini Test 2, Assignment) to visualize the spread and detect potential outliers in scores.
4.	Create a chart to show the students where attendance is less than 50%.
5.	Any other visualization/analysis which you can infer from data for the management.


In [ ]:
bar_df = student_ma_merged.sort_values(by="weighted percentage",ascending=False).head(5)
bar_df

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 6))  
ax = sns.barplot(data=bar_df, x="Name", y="weighted percentage")

for p in ax.patches:
    ax.annotate(f'{p.get_height():.1f}%',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center',
                xytext=(0, 9),
                textcoords='offset points')
plt.title("Top 5 Students' Weighted Percentages", fontsize=16)
plt.xlabel("Student Name", fontsize=12)
plt.ylabel("Weighted Percentage (%)", fontsize=12)
plt.show()

In [ ]:
pie_chart=student_ma_merged["performance"].value_counts()
plt.pie(pie_chart, labels=pie_chart.index, autopct='%1.1f%%', startangle=180)
plt.axis('equal')
plt.title('Performace of Students')
plt.show()

In [ ]:
a=["Mini Test 1","Mini Test 2","Live Test","Assignment"]

In [ ]:
for i in a:
    # Set the plot style for a clean, professional look.
    sns.set_theme(style="whitegrid")
    
    # Create a new figure and axes for each plot.
    plt.figure(figsize=(8, 6))

    # Create the box plot for the single column 'i'.
    plt.boxplot(x=student_ma_merged[i])

    # Add a clear title and descriptive labels as requested.
    plt.title(f"Box Plot for {i}", fontsize=16)
    plt.xlabel("Marks", fontsize=12)
    plt.ylabel(i, fontsize=12)

    # Display the plot.
    plt.show()


In [ ]:
hist_50=student_ma_merged.loc[(student_ma_merged["Attendance percentage"]<50),["Attendance percentage","Name"]]
hist_50

In [ ]:

sns.set_theme(style="whitegrid")

# Create a new figure and axes for the plot.
fig, ax = plt.subplots(figsize=(10, 6))


sns.histplot(
    data=hist_50,
    x='Attendance percentage',
    binwidth=10
)


ax.bar_label(ax.containers[0])


plt.title("Distribution of Attendance Percentage", fontsize=16)
plt.xlabel("Attendance Percentage", fontsize=12)
plt.ylabel("Frequency", fontsize=12)

# Display the plot.
plt.show()


#### Other Charts

In [ ]:
selected_data = student_ma_merged[['weighted percentage', 'Total_Attendance']]
correlation_matrix = selected_data.corr()
sns.set_theme(style="white")
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("Correlation Heatmap: Weighted Percentage vs. Total Attendance", fontsize=14)
plt.show()

In [ ]:
bar_df_lowest = student_ma_merged.sort_values(by="weighted percentage").head(10)
bar_df_lowest

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 6))  
ax = sns.barplot(data=bar_df_lowest, x="Name", y="weighted percentage")

for p in ax.patches:
    ax.annotate(f'{p.get_height():.1f}%',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center',
                xytext=(0, 9),
                textcoords='offset points')
plt.title("Lowest 10 Students' Weighted Percentages", fontsize=16)
plt.xlabel("Student Name", fontsize=12)
plt.ylabel("Weighted Percentage (%)", fontsize=12)
plt.show()

In [ ]:
hist_50_less=student_ma_merged.loc[(student_ma_merged["Attendance percentage"]<30),["Attendance percentage","Name"]]
hist_50_less

In [ ]:

sns.set_theme(style="whitegrid")

# Create a new figure and axes for the plot.
fig, ax = plt.subplots(figsize=(10, 6))


sns.histplot(
    data=hist_50_less,
    x='Attendance percentage',
    binwidth=5
)


ax.bar_label(ax.containers[0])


plt.title("Distribution of  Lowest Attendance Percentage", fontsize=16)
plt.xlabel("Attendance Percentage", fontsize=12)
plt.ylabel("Frequency", fontsize=12)

# Display the plot.
plt.show()